In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
#Staff
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [6]:
staff_data_df.rename(columns={
    'ID_Staff':'ID_Staff__c',
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate' : 'BirthDate__c'
}, inplace=True)
staff_data_df.head()

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [7]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'LastName__c': 'Wimberly',
  'FirstName__c': 'Sam',
  'MiddleName__c': 'Nico',
  'BirthDate__c': datetime.date(1995, 7, 5)},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': datetime.date(1992, 4, 22)},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': datetime.date(1993, 5, 21)},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'LastName__c': 'Popelka',
  'FirstName__c': 'Sarah',
  'MiddleName__c': 'Nisan',
  'BirthDate__c': datetime.date(1994, 6, 7)}]

In [8]:
for rec in staff_data_records:

    record = {
        'ID_Staff__c': rec['ID_Staff__c'],
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': rec['BirthDate__c'].isoformat(),
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

In [9]:
staff_lookup_list = []

data = sf.query_all_iter("SELECT ID_Staff__c, Name FROM Staff__c")
for row in data:
    rec = {
        'ID_Staff__c': row['ID_Staff__c'],
        'Staff__c': row['Name']
    }
    staff_lookup_list.append(rec)

In [10]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_Staff__c,Staff__c
0,1.0,a0F3h000000NatL
1,4.0,a0F3h000000Nata
2,2.0,a0F3h000000NatQ
3,3.0,a0F3h000000NatV


In [11]:
class_lookup_list = []

data = sf.query_all_iter("SELECT ID_Class__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class__c': row['ID_Class__c'],
        'Class_ID_FK__c': row['Name']
    }
    class_lookup_list.append(rec)

In [12]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,ID_Class__c,Class_ID_FK__c
0,2.0,a0H3h000003S9e5
1,3.0,a0H3h000003S9eA
2,4.0,a0H3h000003S9eF
3,1.0,a0H3h000003S9e0


In [13]:
#Staffassignment
query = '''
    SELECT
        sa.*
        
    FROM
        staffassignment sa
        
'''
staffassignment_data_df = pd.read_sql(query, conn)
staffassignment_data_df.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [14]:
staffassignment_data_df.rename(columns={
    'ID_StaffAssignment' : 'ID_StaffAssignment__c',
    'ID_Staff': 'ID_Staff__c',
    'ID_Class':'ID_Class__c',
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)

staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [15]:
staffassignment_data_df = pd.merge(staffassignment_data_df, staff_lookup_df, how='left')
staffassignment_data_df.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Staff__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a0F3h000000NatL
1,2,4,1,Teacher Assistant,2020-03-16,None,a0F3h000000Nata
2,3,3,1,Instructor,2020-03-16,None,a0F3h000000NatV


In [16]:
staffassignment_data_df = pd.merge(staffassignment_data_df, class_lookup_df, how='left')
staffassignment_data_df.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Staff__c,Class_ID_FK__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a0F3h000000NatL,a0H3h000003S9e0
1,2,4,1,Teacher Assistant,2020-03-16,None,a0F3h000000Nata,a0H3h000003S9e0
2,3,3,1,Instructor,2020-03-16,None,a0F3h000000NatV,a0H3h000003S9e0


In [17]:
staffassignment_data_records = staffassignment_data_df.to_dict(orient='records')
staffassignment_data_records

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'Staff__c': 'a0F3h000000NatL',
  'Class_ID_FK__c': 'a0H3h000003S9e0'},
 {'ID_StaffAssignment__c': 2,
  'ID_Staff__c': 4,
  'ID_Class__c': 1,
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'Staff__c': 'a0F3h000000Nata',
  'Class_ID_FK__c': 'a0H3h000003S9e0'},
 {'ID_StaffAssignment__c': 3,
  'ID_Staff__c': 3,
  'ID_Class__c': 1,
  'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 3, 16),
  'EndDate__c': None,
  'Staff__c': 'a0F3h000000NatV',
  'Class_ID_FK__c': 'a0H3h000003S9e0'}]

In [18]:
for rec in staffassignment_data_records:
 
    record = {
        'ID_StaffAssignment__c': rec['ID_StaffAssignment__c'],
        'ID_Staff__c': rec['ID_Staff__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'Role__c': rec['Role__c'],
        'StartDate__c': rec['StartDate__c'].isoformat(),
        'EndDate__c': rec['EndDate__c'],
        'Staff_ID__c': rec['Staff__c'],
        'Class_ID_FK__c': rec['Class_ID_FK__c']
    }
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
class_records = sf.query("SELECT Id FROM Staff_Assignment__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

In [ ]:
 sf.bulk.Staff__c.delete(recs_to_delete)